In [1]:
"""
Adapted from https://github.com/huggingface/diffusers/issues/2326 by https://github.com/ignacfetser

The LoRA trained using Diffusers are saved in .bin or .pkl format, which must be converted to be used in Automatic1111 WebUI.

This script converts .bin or .pkl files into .safetensors format, which can be used in WebUI.

Put this file in the same folder of .bin or .pkl file and run `python convert-to-safetensors.py --file checkpoint_file`

"""
import re
import os
import argparse
import torch
from safetensors.torch import save_file


def convert_bin(file_path):
    ## use GPU or CPU
    if torch.cuda.is_available():
        device = "cuda"
        checkpoint = torch.load(file_path, map_location=torch.device("cuda"))
    else:
        device = "cpu"
        # if on CPU or want to have maximum precision on GPU, use default full-precision setting
        checkpoint = torch.load(file_path, map_location=torch.device("cpu"))

    print(f"device is {device}")

    new_dict = dict()
    for idx, key in enumerate(checkpoint):
        new_key = re.sub("\.processor\.", "_", key)
        new_key = re.sub("mid_block\.", "mid_block_", new_key)
        new_key = re.sub("_lora.up.", ".lora_up.", new_key)
        new_key = re.sub("_lora.down.", ".lora_down.", new_key)
        new_key = re.sub("\.(\d+)\.", "_\\1_", new_key)
        new_key = re.sub("to_out", "to_out_0", new_key)
        new_key = "lora_unet_" + new_key

        new_dict[new_key] = checkpoint[key]

    file_name = os.path.splitext(file_path)[
        0
    ]  # get the file name without the extension
    new_lora_name = file_name + ".safetensors"
    print("Saving " + new_lora_name)
    save_file(new_dict, new_lora_name)

In [2]:
# FPATH = "/workspace/dso/GenSAR/LoRA/output/DOSRS_v1/shiprs_512_sd15_lr1e-04/checkpoint-2160/pytorch_model.bin"
# FPATH = "/workspace/dso/GenSAR/LoRA/output/DOSRS_v1/512_sd15_lr1e-04/checkpoint-2700/pytorch_model.bin"
# FPATH = "/workspace/dso/GenSAR/LoRA/output/sarlora256_lr1e-04/checkpoint-10000/pytorch_model.bin"
FPATH = "/workspace/dso/GenSAR/LoRA/output/DOSRS_v2/dosrsv2_512_sd15_lr1e-04/checkpoint-5400/pytorch_model.bin"
convert_bin(FPATH)

device is cuda
Saving /workspace/dso/GenSAR/LoRA/output/DOSRS_v2/dosrsv2_512_sd15_lr1e-04/checkpoint-5400/pytorch_model.safetensors


Bad pipe message: %s [b'"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge"']
Bad pipe message: %s [b'="114"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "macOS"\r']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"\r\nsec-ch-ua-mobile: ?0\r\n']
Bad pipe message: %s [b'c-ch-ua-platform: "macOS"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_', b' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0', b' Safari/537.36 Edg/114.0.1823.58\r\nAccept: tex']
Bad pipe message: %s [b'html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q', b'.7\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetc']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"\r\nsec-ch-ua-mobile: ?0\r\n']
Bad pipe message: %s [b'c-ch-ua-platform: "macOS"\r\nUpgrade-

In [2]:
import random


def split_into_subclasses(data, num_subclasses=16):
    splits = {}
    for category, value in data.items():
        remaining = value
        sub_values = []

        for i in range(num_subclasses - 1):  # Subtract 1 to account for last split
            split_val = random.randint(0, remaining)
            sub_values.append(split_val)
            remaining -= split_val

        sub_values.append(remaining)  # the remaining value goes to the last subclass
        splits[category] = sub_values

    return splits


# Sample data
data = {"Cargo": 3890, "Other": 1710, "Fishing": 814, "Tanker": 315, "Dredger": 242}

splits = split_into_subclasses(data)
for category, sub_values in splits.items():
    print(f"{category}: {sub_values}")

Cargo: [3071, 696, 85, 17, 19, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Other: [967, 458, 125, 88, 23, 9, 14, 16, 3, 1, 4, 2, 0, 0, 0, 0]
Fishing: [676, 102, 9, 26, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tanker: [87, 78, 60, 81, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Dredger: [85, 7, 80, 32, 18, 0, 18, 2, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
import random


def split_category(value, num_splits=16):
    splits = [0] * num_splits
    for _ in range(value):
        splits[random.randint(0, num_splits - 1)] += 1
    return splits


data = {"Cargo": 3890, "Other": 1710, "Fishing": 814, "Tanker": 315, "Dredger": 242}

subclasses = []

# Split each category into 16 subclasses
for category, value in data.items():
    splits = split_category(value)
    if not subclasses:
        subclasses = [[val] for val in splits]
    else:
        for i, val in enumerate(splits):
            subclasses[i].append(val)

# Each element in subclasses now contains representation from each of the five main classes
for idx, sc in enumerate(subclasses):
    print(
        f"Subclass {idx + 1}: Cargo={sc[0]}, Other={sc[1]}, Fishing={sc[2]}, Tanker={sc[3]}, Dredger={sc[4]}"
    )

Subclass 1: Cargo=241, Other=124, Fishing=43, Tanker=18, Dredger=24
Subclass 2: Cargo=229, Other=107, Fishing=50, Tanker=24, Dredger=15
Subclass 3: Cargo=246, Other=103, Fishing=54, Tanker=26, Dredger=19
Subclass 4: Cargo=263, Other=110, Fishing=58, Tanker=16, Dredger=11
Subclass 5: Cargo=276, Other=118, Fishing=45, Tanker=23, Dredger=11
Subclass 6: Cargo=221, Other=101, Fishing=51, Tanker=21, Dredger=15
Subclass 7: Cargo=244, Other=83, Fishing=53, Tanker=15, Dredger=9
Subclass 8: Cargo=248, Other=111, Fishing=56, Tanker=15, Dredger=15
Subclass 9: Cargo=240, Other=87, Fishing=51, Tanker=15, Dredger=12
Subclass 10: Cargo=225, Other=104, Fishing=49, Tanker=24, Dredger=10
Subclass 11: Cargo=266, Other=101, Fishing=53, Tanker=20, Dredger=24
Subclass 12: Cargo=216, Other=116, Fishing=59, Tanker=17, Dredger=21
Subclass 13: Cargo=237, Other=120, Fishing=52, Tanker=22, Dredger=16
Subclass 14: Cargo=251, Other=110, Fishing=47, Tanker=20, Dredger=12
Subclass 15: Cargo=248, Other=107, Fishing=46,